# RLDT


## Step 1: Import the necessary libraries:


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random

In [2]:
num_IOT_devices = 10

voltages_frequencies_IOT = [
    (10e6  , 1.8),
    (20e6  , 2.3),
    (40e6  , 2.7),
    (80e6  , 4.0),
    (160e6 , 5.0),
]

num_MEC_devices = 5

voltages_frequencies_MEC = [
    (1500e6 ,  1.2),
    (1000e6 ,  1.0),
    (750e6, 0.825),
    (600e6, 0.8),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5
num_dag_generations = 10000

devices_path = "./resources/mini_devices.csv"
tasks_path = "./resources/mini_tasks.csv"

## Step 2: Define the environment:


### Step 2.1: Devices


#### _IOT_


In [3]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 8, 16])
    device_info = {
        "id": f"iot {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies": [
            [
                voltages_frequencies_IOT[i]
                for i in np.random.choice(5, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "ISL": np.random.randint(10, 21) / 100,
        "capacitance": [np.random.uniform(2, 3) * 1e-10 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([800, 900,1000]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "acceptableTasks": list(np.random.choice(
            task_kinds, size=np.random.randint(3, 5), replace=False
        )),
        "handleSafeTask": np.random.choice([0, 1], p=[0.1, 0.9]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)


#### _MEC_


In [4]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16, 32, 64])
    device_info = {
        "id":f"mec {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies":[
            [
                voltages_frequencies_MEC[i]
                for i in np.random.choice(4, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([550, 650, 750]) * 1e-3 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "acceptableTasks": list(np.random.choice(
            task_kinds, size=np.random.randint(3, 5), replace=False
        )),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)


#### _CLOUD_


In [5]:
device_info = [
    {
        "id": 'cloud',
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [2.8e9, 3.9e9],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "acceptableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)

#### ALL THE DEVICES


In [6]:
devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
devices.to_csv(devices_path)


### Step 2.2: Application


#### _ALL THE TASKS_


In [7]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):

        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(
                available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}")
                           for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

In [8]:
tasks_data = []
start_node_number = 1
for run in range(num_dag_generations):
    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)
    random_dag = generate_random_dag(num_nodes)
    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }
    random_dag = nx.relabel_nodes(random_dag, mapping)
    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "job": run,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": 5*1e6,
            "dataEntrySize":5*1e6,
            "returnDataSize":5*1e6,
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes
np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)
tasks = tasks.set_index("id")
tasks.to_csv(tasks_path)